In [19]:
# Cell: Imports + utilities + setup
import os
import pathlib
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from typing import Dict, List, Any
import datetime as dt
import urllib.robotparser

# create data/raw if not exists
DATA_RAW = pathlib.Path("data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

# load .env (must run before using ALPHA_KEY)
load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded ALPHAVANTAGE_API_KEY?", bool(ALPHA_KEY))

# --- safe timestamp & filename helpers ---
def safe_stamp() -> str:
    # UTC timestamp (no spaces), safe for filenames
    return dt.datetime.utcnow().strftime("%Y%m%d_%H%M%S")

def safe_filename(prefix: str, meta: Dict[str, Any]) -> str:
    # meta is a dict; sanitize values (no spaces), limit lengths
    mid = "_".join([f"{k}-{str(v).replace(' ', '-')[:20]}" for k, v in meta.items()])
    return f"{prefix}_{mid}_{safe_stamp()}.csv"

# --- improved validate_df (structured messages) ---
def validate_df(df: pd.DataFrame, required_cols: List[str], dtypes_map: Dict[str, str]) -> Dict[str, str]:
    msgs = {}
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        msgs['missing_cols'] = f"Missing columns: {missing}"
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                if dtype == 'datetime64[ns]':
                    pd.to_datetime(df[col])
                elif dtype == 'float':
                    pd.to_numeric(df[col])
            except Exception as e:
                msgs[f'dtype_{col}'] = f"Failed to coerce {col} to {dtype}: {e}"
    na_counts = df.isna().sum().sum()
    msgs['na_total'] = f"Total NA values: {na_counts}"
    return msgs

Loaded ALPHAVANTAGE_API_KEY? True


In [20]:
use_alpha = bool(ALPHA_KEY)
import yfinance as yf
SYMBOL='AAPL'
df_api = yf.download(SYMBOL, period="6mo", interval="1d", progress=False).reset_index()[['Date','Close']]
df_api.columns = ['date','adj_close']
df_api['date'] = pd.to_datetime(df_api['date'])
df_api['adj_close'] = pd.to_numeric(df_api['adj_close'], errors='coerce')

# final clean
df_api = df_api.sort_values('date').reset_index(drop=True)

# validate
msgs_api = validate_df(df_api, required_cols=['date','adj_close'],
                       dtypes_map={'date':'datetime64[ns]','adj_close':'float'})
print("API validation:", msgs_api)
fname = safe_filename(prefix="api", meta={"source":  "yfinance", "symbol": SYMBOL})
out_path = DATA_RAW / fname
df_api.to_csv(out_path, index=False)
print("Saved:", out_path)

C:\Users\Bobli\AppData\Local\Temp\ipykernel_42736\3310172685.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period="6mo", interval="1d", progress=False).reset_index()[['Date','Close']]


API validation: {'na_total': 'Total NA values: 0'}
Saved: data\raw\api_source-yfinance_symbol-AAPL_20250818_005616.csv


In [21]:
SCRAPE_URL = "https://www.w3schools.com/html/html_tables.asp"  # replace with permitted page
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"}

resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, 'html.parser')
table = soup.find('table')
rows = []
for tr in table.find_all('tr'):
    cells = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
    if cells:
        rows.append(cells)
    # assume first row is header
print(rows)
header, *data = rows
df_scrape = pd.DataFrame(data, columns=header) 


[['Company', 'Contact', 'Country'], ['Alfreds Futterkiste', 'Maria Anders', 'Germany'], ['Centro comercial Moctezuma', 'Francisco Chang', 'Mexico'], ['Ernst Handel', 'Roland Mendel', 'Austria'], ['Island Trading', 'Helen Bennett', 'UK'], ['Laughing Bacchus Winecellars', 'Yoshi Tannamuri', 'Canada'], ['Magazzini Alimentari Riuniti', 'Giovanni Rovelli', 'Italy']]


In [22]:
msgs2 = validate_df(df_scrape, required_cols=list(df_scrape.columns), dtypes_map={})
print(msgs2)

fname2 = safe_filename(prefix="scrape", meta={"site": "example", "table": "markets"})
out_path2 = DATA_RAW / fname2
df_scrape.to_csv(out_path2, index=False)
print("Saved:", out_path2)

{'na_total': 'Total NA values: 0'}
Saved: data\raw\scrape_site-example_table-markets_20250818_005648.csv


# Data acquisition — sources & validation (fill-in)

## Data sources
- API:
  - Source: AlphaVantage (if API key provided) OR yfinance fallback.
  - Endpoint / ticker: SYMBOL = AAPL  (change if needed).
  - Params used: function=TIME_SERIES_DAILY_ADJUSTED, outputsize=compact (AlphaVantage) / period=6mo (yfinance)

- Scrape:
  - Scrape URL: <PUT YOUR SCRAPE_URL HERE>
  - Table: first `<table>` on page (you may choose a more precise selector).
  - Request headers: custom User-Agent "AFE-Course-Notebook/1.0 (contact: instructor@example.edu)"
  - Robots.txt: checked programmatically before scraping.

## Validation logic
- Required columns: API must have `date` and `adj_close`. Scrape requires the columns found in the table.
- Dtype checks: `date` -> datetime64[ns]; `adj_close` / price-like columns -> numeric (float).
- NA checks: report total NA counts for early diagnosis.
- Shape: saved for record in validation info.

## Files saved
- Saved raw CSVs in `data/raw/`:
  - `<api_...>.csv`
  - `<scrape_...>.csv`

## .env & secrets
- `.env` (contains ALPHAVANTAGE_API_KEY) **MUST NOT** be committed to Git.
- Add `.env` to `.gitignore`.

## Assumptions & Risks
- Assumes AlphaVantage returns a standard "Time Series" JSON structure. Risk: rate limiting (AlphaVantage returns 'Note' on limit).
- Assumes the scraped page allows crawling via robots.txt and that the first `<table>` is the desired data; risk: page layout differences or nested tables.
- Demo fallback is included to ensure the notebook runs even if the external sources are blocked.
